In [1]:
#import pandas,numpy,xgboost,datetime
import pandas as pd
import numpy as np
import xgboost as xgb
from datetime import datetime,timedelta

In [2]:
def AddDatetime(df):
    df["dt"]=dt=pd.to_datetime(str(2016)+"-"+df["month"].astype(str)+"-"+df["date"].astype(str)\
            +" "+df["hour"].astype(str)+":"+df["minute"].astype(str))
    df["dt"]=df["dt"].dt.dayofyear*72+df["time_number"]
    return df

In [3]:
def mape_nparray(y,d):
    c=d
    rec=np.sum(np.abs(y-c)/c)/len(c)
    return rec

def mape_nparray_ln(y,d):
    c=d
    result=np.sum(np.abs(np.power(np.e,y)-np.abs(np.power(np.e,c)))/np.abs(np.power(np.e,c)))/len(c)
    return result

def mape(y,d):
    c=d.get_label()
    result=np.sum(np.abs(y-c)/c)/len(c)
    return "mape",result

def mape_ln(y,d):
    c=d.get_label()
    result=np.sum(np.abs(np.power(np.e,y)-np.abs(np.power(np.e,c)))/np.abs(np.power(np.e,c)))/len(c)
    return "mape",result

def mape_object(y,d):
    g=1.0*np.sign(y-d)/d
    h=1.0/d
    return -g,h

In [4]:
def DataAverage(df):
    q=df.groupby(["month","date","hour","minute","intersection_id","tollgate_id"])["travel_time"].mean()
    p=df.drop(["travel_time","datetime","starting_time"],axis=1).drop_duplicates().sort(columns=["month","date","hour","minute","intersection_id","tollgate_id"])
    p["travel_time"]=q.values
    cols=list(p)
    cols.insert(2, cols.pop(cols.index('travel_time')))
    p= p.ix[:, cols]
    return p

In [5]:
#Use KNN Algorithm to select validation data(10-13,10-13,9-29,9-26,10-2,9-25,-10-6)

def SelectTrainAndValidation(data):
    select_time2=((data["hour"]<10)&(data["hour"]>7))|((data["hour"]<19)&(data["hour"]>16))
    select_time1=((data["hour"]<8)&(data["hour"]>5))|((data["hour"]<17)&(data["hour"]>14))
    
    t1=(data["month"]==10)&(data["date"]==13)&select_time2
    t2=(data["month"]==10)&(data["date"]==13)&select_time2
    t3=(data["month"]==9)&(data["date"]==29)&select_time2
    t4=(data["month"]==9)&(data["date"]==26)&select_time2
    t5=(data["month"]==10)&(data["date"]==2)&select_time2
    t6=(data["month"]==9)&(data["date"]==25)&select_time2
    t7=(data["month"]==10)&(data["date"]==6)&select_time2
    
    e1=(data["month"]==10)&(data["date"]==13)&select_time1
    e2=(data["month"]==10)&(data["date"]==13)&select_time1
    e3=(data["month"]==9)&(data["date"]==29)&select_time1
    e4=(data["month"]==9)&(data["date"]==26)&select_time1
    e5=(data["month"]==10)&(data["date"]==22)&select_time1
    e6=(data["month"]==9)&(data["date"]==25)&select_time1
    e7=(data["month"]==10)&(data["date"]==6)&select_time1
    
    dt1=data[t1].copy()
    dt2=data[t2].copy()
    dt3=data[t3].copy()
    dt4=data[t4].copy()
    dt5=data[t5].copy()
    dt6=data[t6].copy()
    dt7=data[t7].copy()
    
    
    de1=data[e1].copy()
    de2=data[e2].copy()
    de3=data[e3].copy()
    de4=data[e4].copy()
    de5=data[e5].copy()
    de6=data[e6].copy()
    de7=data[e7].copy()

    
    dtrain=data[~(t1|t2|t3|t4|t5|t6|t7|e1|e2|e3|e4|e5|e6|e7)].copy()
    dpred=pd.concat([dt1,dt2,dt3,dt4,dt5,dt6,dt7])
    dtest=pd.concat([de1,de2,de3,de4,de5,de6,de7])
    return dtrain,dtest,dpred,data

In [6]:
def AddFeature_target(data): 
    
    #month,date,hour,minute
    data_starting_time=data["starting_time"].str.split(' ',expand=True)
    data_starting_time_date=data_starting_time[0].str.split('-',expand=True)
    data_starting_time_time=data_starting_time[1].str.split(':',expand=True)
    data["month"]=data_starting_time_date[1].apply(int)
    data["date"]=data_starting_time_date[2].apply(int)
    data["hour"]=data_starting_time_time[0].apply(int)
    data["minute"]=data_starting_time_time[1].apply(int)
    data["minute"]=(data["minute"]/20).apply(int)*20
    data["number_intersection_id"]=data["intersection_id"].replace(["A","B","C"],[1,2,3])
    
    #holiday
    h1=data[data["starting_time"]<"2016-09-15"]
    h2=data[(data["starting_time"]>"2016-09-15")&(data["starting_time"]<"2016-09-18")]
    h3=data[(data["starting_time"]>"2016-09-18")&(data["starting_time"]<"2016-09-30")]
    h4=data[(data["starting_time"]>"2016-09-30")&(data["starting_time"]<"2016-10-08")]
    h5=data[data["starting_time"]>"2016-10-08"]
    h1["holiday"]=0
    h2["holiday"]=1
    h3["holiday"]=0
    h4["holiday"]=1
    h5["holiday"]=0
    data=pd.concat([h1,h2,h3,h4,h5])
    
    #datetime,weekday
    data["datetime"]=pd.to_datetime(data["starting_time"])
    data["weekday"]=data["datetime"].dt.weekday
    
    #one_hot A,B,C
    data["ifA"]=data["intersection_id"].replace(["A","B","C"],[1,0,0])
    data["ifB"]=data["intersection_id"].replace(["A","B","C"],[0,1,0])
    data["ifC"]=data["intersection_id"].replace(["A","B","C"],[0,0,1])

    #one_hot 1,2,3
    data["if1"]=data["tollgate_id"].replace([1,2,3],[1,0,0])
    data["if2"]=data["tollgate_id"].replace([1,2,3],[0,1,0])
    data["if3"]=data["tollgate_id"].replace([1,2,3],[0,0,1])
    
    #one_hot weekday
    data["wk1"]=data["weekday"].replace(range(0,7),[1,0,0,0,0,0,0])
    data["wk2"]=data["weekday"].replace(range(0,7),[0,1,0,0,0,0,0])
    data["wk3"]=data["weekday"].replace(range(0,7),[0,0,1,0,0,0,0])
    data["wk4"]=data["weekday"].replace(range(0,7),[0,0,0,1,0,0,0])
    data["wk5"]=data["weekday"].replace(range(0,7),[0,0,0,0,1,0,0])
    data["wk6"]=data["weekday"].replace(range(0,7),[0,0,0,0,0,1,0])
    data["wk7"]=data["weekday"].replace(range(0,7),[0,0,0,0,0,0,1])
    
    #time_number,date_number
    data["time_number"]=((data["hour"]*3+data["minute"]/20)).astype(int)
    data["date_number"]=data["month"]*31+data["date"]
    data=data.iloc[:,[0,1,2]+range(3,len(data.T))]
    return data

In [7]:
def AddFeature_rawdata(data):
      
    #month,date,hour,minute
    data_starting_time=data["starting_time"].str.split(' ',expand=True)
    data_starting_time_date=data_starting_time[0].str.split('-',expand=True)
    data_starting_time_time=data_starting_time[1].str.split(':',expand=True)
    data["month"]=data_starting_time_date[1].apply(int)
    data["date"]=data_starting_time_date[2].apply(int)
    data["hour"]=data_starting_time_time[0].apply(int)
    data["minute"]=data_starting_time_time[1].apply(int)
    data["minute"]=(data["minute"]/20).apply(int)*20
    data["number_intersection_id"]=data["intersection_id"].replace(["A","B","C"],[1,2,3])
    
    #holiday
    h1=data[(data["starting_time"]<"2016-09-15")]
    h2=data[(data["starting_time"]>"2016-09-15")&(data["starting_time"]<"2016-09-18")]
    h3=data[(data["starting_time"]>"2016-09-18")&(data["starting_time"]<"2016-09-30")]
    h4=data[(data["starting_time"]>"2016-09-30")&(data["starting_time"]<"2016-10-08")]
    h5=data[data["starting_time"]>"2016-10-08"]
    h1["holiday"]=0
    h2["holiday"]=1
    h3["holiday"]=0
    h4["holiday"]=1
    h5["holiday"]=0
    data=pd.concat([h1,h2,h3,h4,h5])
    
    #datetime,weekday
    data["datetime"]=pd.to_datetime(data["starting_time"])
    data["weekday"]=data["datetime"].dt.weekday
    
    #one_hot A,B,C
    data["ifA"]=data["intersection_id"].replace(["A","B","C"],[1,0,0])
    data["ifB"]=data["intersection_id"].replace(["A","B","C"],[0,1,0])
    data["ifC"]=data["intersection_id"].replace(["A","B","C"],[0,0,1])

    #one_hot 1,2,3
    data["if1"]=data["tollgate_id"].replace([1,2,3],[1,0,0])
    data["if2"]=data["tollgate_id"].replace([1,2,3],[0,1,0])
    data["if3"]=data["tollgate_id"].replace([1,2,3],[0,0,1])
    
    #one_hot weekday
    data["wk1"]=data["weekday"].replace(range(0,7),[1,0,0,0,0,0,0])
    data["wk2"]=data["weekday"].replace(range(0,7),[0,1,0,0,0,0,0])
    data["wk3"]=data["weekday"].replace(range(0,7),[0,0,1,0,0,0,0])
    data["wk4"]=data["weekday"].replace(range(0,7),[0,0,0,1,0,0,0])
    data["wk5"]=data["weekday"].replace(range(0,7),[0,0,0,0,1,0,0])
    data["wk6"]=data["weekday"].replace(range(0,7),[0,0,0,0,0,1,0])
    data["wk7"]=data["weekday"].replace(range(0,7),[0,0,0,0,0,0,1])
    data["time_number"]=((data["hour"]*3+data["minute"]/20)).astype(int)
    data["date_number"]=data["month"]*31+data["date"]
    data=data.iloc[:,[0,1,2]+range(3,len(data.T))]
        
    datatrain,datatest,datapredict,datatransforemed=SelectTrainAndValidation(data)
    datavalidation=datapredict
    q=datavalidation.groupby(["month","date","hour","minute","intersection_id","tollgate_id"])["travel_time"].mean()
    p=datavalidation.drop(["travel_time","datetime","starting_time"],axis=1).drop_duplicates().sort(columns=["month","date","hour","minute","intersection_id","tollgate_id"])
    p["travel_time"]=q.values
    cols=list(p)
    cols.insert(2, cols.pop(cols.index('travel_time')))
    p= p.ix[:, cols]
    datavalidation=p    

    return datatrain,datatest,datavalidation,datatransforemed

In [8]:
#read data
data1 = pd.read_csv('../RawData/trajectories(table_5)_training2.csv')
data2 = pd.read_csv('../RawData/trajectories(table 5)_test2.csv')
data3 = pd.read_csv('../RawData/trajectories(table 5)_training.csv')

#get training data,validation data
data=pd.concat([data1,data3])
data=data.iloc[:,[0,1,3,5]]
datatrain,datatest,datavalidation,datatransforemed=AddFeature_rawdata(data)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [9]:
datavalidation=pd.concat([datavalidation,datavalidation[datavalidation["date"]==13]])

In [10]:
train=pd.concat([datatrain,datatest])
t=DataAverage(train.copy())
train=pd.concat([t,DataAverage(AddFeature_target(data2).ix[:,list(datatrain)])])

HIGH_TRAIN=500
LOW_TRAIN=10
train=train[(train["travel_time"]>LOW_TRAIN)&(train["travel_time"]<HIGH_TRAIN)]

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


In [11]:
# Select Feature
feature=[2]+range(10,16)+[8,9]+[3,-2]+range(16,23)

x=train.copy().iloc[:,feature]
xx=x.copy().drop(['travel_time'],axis=1)
xy=np.log(x['travel_time'])

z=datavalidation.copy().iloc[:,feature]
zx=z.drop(['travel_time'],axis=1)
zy=np.log(z['travel_time'])
z

,travel_time,ifA,ifB,ifC,if1,if2,if3,holiday,weekday,month,time_number,wk1,wk2,wk3,wk4,wk5,wk6,wk7
68473,52.721765,1,0,0,0,1,0,0,6,9,24,0,0,0,0,0,0,1
68471,113.111818,1,0,0,0,0,1,0,6,9,24,0,0,0,0,0,0,1
68469,128.096000,0,1,0,1,0,0,0,6,9,24,0,0,0,0,0,0,1
68470,55.525000,0,1,0,0,0,1,0,6,9,24,0,0,0,0,0,0,1
68498,123.630000,0,0,1,1,0,0,0,6,9,24,0,0,0,0,0,0,1
68489,158.455000,0,0,1,0,0,1,0,6,9,24,0,0,0,0,0,0,1
68515,73.422727,1,0,0,0,1,0,0,6,9,25,0,0,0,0,0,0,1
68514,150.573333,1,0,0,0,0,1,0,6,9,25,0,0,0,0,0,0,1
68519,110.823333,0,1,0,1,0,0,0,6,9,25,0,0,0,0,0,0,1
68513,91.273333,0,1,0,0,0,1,0,6,9,25,0,0,0,0,0,0,1


In [12]:
#Train in Xgboost
xlf = xgb.XGBRegressor(max_depth=11, 
                        learning_rate=0.1,
                        n_estimators=4000, 
                        silent=True, 
                        objective="reg:linear", 
    
                        gamma=0,
                        min_child_weight=5, 
                        max_delta_step=0, 
                        subsample=0.8, 
                        colsample_bytree=0.9, 
                        colsample_bylevel=1, 
                        reg_alpha=1e0, 
                        reg_lambda=0, 
                        scale_pos_weight=1, 
                        seed=1, 
                        missing=None)
xlf.fit(xx,xy, eval_metric=mape_ln,verbose = True, eval_set = [(zx, zy)],early_stopping_rounds=20)
limit=xlf.best_iteration+1

[0]	validation_0-rmse:3.91871	validation_0-mape:0.977884
Multiple eval metrics have been passed: 'validation_0-mape' will be used for early stopping.

Will train until validation_0-mape hasn't improved in 20 rounds.
[1]	validation_0-rmse:3.53456	validation_0-mape:0.967808
[2]	validation_0-rmse:3.18888	validation_0-mape:0.954812
[3]	validation_0-rmse:2.87836	validation_0-mape:0.938642
[4]	validation_0-rmse:2.59898	validation_0-mape:0.919145
[5]	validation_0-rmse:2.34683	validation_0-mape:0.896282
[6]	validation_0-rmse:2.12088	validation_0-mape:0.870136
[7]	validation_0-rmse:1.92077	validation_0-mape:0.841564
[8]	validation_0-rmse:1.73865	validation_0-mape:0.809942
[9]	validation_0-rmse:1.57446	validation_0-mape:0.775907
[10]	validation_0-rmse:1.42764	validation_0-mape:0.74021
[11]	validation_0-rmse:1.29449	validation_0-mape:0.702643
[12]	validation_0-rmse:1.17614	validation_0-mape:0.664381
[13]	validation_0-rmse:1.0712	validation_0-mape:0.626135
[14]	validation_0-rmse:0.978635	validatio

# OUT

In [17]:
target["tw"]

KeyError: 'tw'

In [18]:
df_id=pd.DataFrame({"intersection_id":["A","A","B","B","C","C"],\
             "tollgate_id":[2,3,1,3,1,3]})
tmptime=datetime(2016,10,25)
timeseries=[]
while(tmptime<datetime(2016,11,1)):
    if((tmptime.hour==8)|(tmptime.hour==9)|(tmptime.hour==17)|(tmptime.hour==18)):
        timeseries=timeseries+[str(tmptime)]
    tmptime=tmptime+timedelta(minutes=20)
df_tw=pd.DataFrame({"tw":timeseries})

In [25]:
df_id=pd.DataFrame({"intersection_id":["A","A","B","B","C","C"],\
             "tollgate_id":[2,3,1,3,1,3]})
tmptime=datetime(2016,10,25)
timeseries=[]
timeseries2=[]
while(tmptime<datetime(2016,11,1)):
    if((tmptime.hour==8)|(tmptime.hour==9)|(tmptime.hour==17)|(tmptime.hour==18)):
        timeseries=timeseries+[str(tmptime)]
        timeseries2=timeseries2+[str(tmptime+timedelta(minutes=20))]
    tmptime=tmptime+timedelta(minutes=20)
df_tw=pd.DataFrame({"tw":timeseries})
df_tw2=pd.DataFrame({"tw":timeseries2})

In [26]:
df_tw2

,tw
0,2016-10-25 08:20:00
1,2016-10-25 08:40:00
2,2016-10-25 09:00:00
3,2016-10-25 09:20:00
4,2016-10-25 09:40:00
5,2016-10-25 10:00:00
6,2016-10-25 17:20:00
7,2016-10-25 17:40:00
8,2016-10-25 18:00:00
9,2016-10-25 18:20:00


In [31]:
# Predict
df_id=pd.DataFrame({"intersection_id":["A","A","B","B","C","C"],\
             "tollgate_id":[2,3,1,3,1,3]})
tmptime=datetime(2016,10,25)
timeseries=[]
timeseries2=[]
while(tmptime<datetime(2016,11,1)):
    if((tmptime.hour==8)|(tmptime.hour==9)|(tmptime.hour==17)|(tmptime.hour==18)):
        timeseries=timeseries+[str(tmptime)]
        timeseries2=timeseries2+[str(tmptime+timedelta(minutes=20))]
    tmptime=tmptime+timedelta(minutes=20)
df_tw=pd.DataFrame({"tw":timeseries,"tw2":timeseries2})

df_tw["merge"]=1
df_id["merge"]=1
target=pd.merge(df_tw,df_id,on=["merge"])
target["time_window"]="["+target["tw"].astype(str)+","+target["tw2"].astype(str)+")"
target

,tw,tw2,merge,intersection_id,tollgate_id,time_window
0,2016-10-25 08:00:00,2016-10-25 08:20:00,1,A,2,"[2016-10-25 08:00:00,2016-10-25 08:20:00)"
1,2016-10-25 08:00:00,2016-10-25 08:20:00,1,A,3,"[2016-10-25 08:00:00,2016-10-25 08:20:00)"
2,2016-10-25 08:00:00,2016-10-25 08:20:00,1,B,1,"[2016-10-25 08:00:00,2016-10-25 08:20:00)"
3,2016-10-25 08:00:00,2016-10-25 08:20:00,1,B,3,"[2016-10-25 08:00:00,2016-10-25 08:20:00)"
4,2016-10-25 08:00:00,2016-10-25 08:20:00,1,C,1,"[2016-10-25 08:00:00,2016-10-25 08:20:00)"
5,2016-10-25 08:00:00,2016-10-25 08:20:00,1,C,3,"[2016-10-25 08:00:00,2016-10-25 08:20:00)"
6,2016-10-25 08:20:00,2016-10-25 08:40:00,1,A,2,"[2016-10-25 08:20:00,2016-10-25 08:40:00)"
7,2016-10-25 08:20:00,2016-10-25 08:40:00,1,A,3,"[2016-10-25 08:20:00,2016-10-25 08:40:00)"
8,2016-10-25 08:20:00,2016-10-25 08:40:00,1,B,1,"[2016-10-25 08:20:00,2016-10-25 08:40:00)"
9,2016-10-25 08:20:00,2016-10-25 08:40:00,1,B,3,"[2016-10-25 08:20:00,2016-10-25 08:40:00)"


In [32]:
# Predict
df_id=pd.DataFrame({"intersection_id":["A","A","B","B","C","C"],\
             "tollgate_id":[2,3,1,3,1,3]})
tmptime=datetime(2016,10,25)
timeseries=[]
timeseries2=[]
while(tmptime<datetime(2016,11,1)):
    if((tmptime.hour==8)|(tmptime.hour==9)|(tmptime.hour==17)|(tmptime.hour==18)):
        timeseries=timeseries+[str(tmptime)]
        timeseries2=timeseries2+[str(tmptime+timedelta(minutes=20))]
    tmptime=tmptime+timedelta(minutes=20)
df_tw=pd.DataFrame({"tw":timeseries,"tw2":timeseries2})

df_tw["merge"]=1
df_id["merge"]=1
target=pd.merge(df_tw,df_id,on=["merge"])
target["time_window"]="["+target["tw"].astype(str)+","+target["tw2"].astype(str)+")"
target=target.iloc[:,[3,4,5]]

target=target.sort(columns=["intersection_id","tollgate_id","time_window"])
target["avg_travel_time"]=1

time_window=target["time_window"].copy()
target["time_window"]=target["time_window"].str.split(',',expand=True)[0]
target["time_window"]=target["time_window"].str.split('[',expand=True)[1]
target=target.rename(columns={"time_window":"starting_time","avg_travel_time":"travel_time"})
target=AddFeature_target(target)
target=target.sort(columns=["intersection_id","tollgate_id","starting_time"])

target.index=range(0,len(target))
target["time_window"]=time_window.values
targetx=target.ix[:,list(datavalidation)].copy().iloc[:,feature].drop(['travel_time'],axis=1)
target["travel_time"]=np.power(np.e,xlf.predict(targetx,ntree_limit=limit))
target=target.rename(columns={"travel_time":"avg_travel_time"})

target1=target[(target["hour"]>0)&(target["hour"]<12)].\
                sort(columns=["intersection_id","tollgate_id","date","hour","minute"])
target2=target[(target["hour"]>12)&(target["hour"]<24)].\
                sort(columns=["intersection_id","tollgate_id","date","hour","minute"])
predict=pd.concat([target1,target2])
predict.index=range(0,len(predict))
predict=predict.iloc[:,[0,1,-1,3]]
predict.to_csv("../Submisssion/submission.csv",index=False)
predict

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:19: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:27: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:35: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:36: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,intersection_id,tollgate_id,time_window,avg_travel_time
0,A,2,"[2016-10-25 08:00:00,2016-10-25 08:20:00)",71.143562
1,A,2,"[2016-10-25 08:20:00,2016-10-25 08:40:00)",72.583389
2,A,2,"[2016-10-25 08:40:00,2016-10-25 09:00:00)",72.433502
3,A,2,"[2016-10-25 09:00:00,2016-10-25 09:20:00)",71.641533
4,A,2,"[2016-10-25 09:20:00,2016-10-25 09:40:00)",71.641533
5,A,2,"[2016-10-25 09:40:00,2016-10-25 10:00:00)",68.387894
6,A,2,"[2016-10-26 08:00:00,2016-10-26 08:20:00)",75.634995
7,A,2,"[2016-10-26 08:20:00,2016-10-26 08:40:00)",76.455063
8,A,2,"[2016-10-26 08:40:00,2016-10-26 09:00:00)",76.432068
9,A,2,"[2016-10-26 09:00:00,2016-10-26 09:20:00)",74.306107
